In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('Data/gold_futures_ohlcv.csv', parse_dates=['ts_event'])
df["ts_event"] = pd.to_datetime(df["ts_event"]).dt.tz_convert('UTC') # Ensure it's UTC

In [3]:
# filter for vanilla futures contracts
single_contract_filter = (df["symbol"].str.len() == 4)
fdf = df[single_contract_filter].copy()

In [4]:
future_month_map = {
    'F': 1, 'G': 2, 'H': 3, 'J': 4, 'K': 5, 'M': 6,
    'N': 7, 'Q': 8, 'U': 9, 'V': 10, 'X': 11, 'Z': 12
}

In [5]:
# function to determine contract year
def get_contract_year(row):
    contract_year_first_digit = int(row["symbol"][3])
    curr_year = row['ts_event'].year
    if contract_year_first_digit < curr_year % 10:
        contract_year = math.ceil(curr_year / 10) * 10 + contract_year_first_digit
    else:
        contract_year = math.floor(curr_year / 10) * 10 + contract_year_first_digit
    return contract_year

fdf["contract_year"] = fdf.apply(get_contract_year, axis=1)
fdf["contract_month"] = fdf.apply(lambda row: future_month_map[row['symbol'][2]], axis=1)

In [6]:
# function to determine expiry length
def get_expiry_length(row):
    month = future_month_map[row['symbol'][2]]
    year = row["contract_year"]
    # Calculate the difference in months
    return (year - row['ts_event'].year) * 12 + month - row['ts_event'].month

fdf["expiry_length"] = fdf.apply(get_expiry_length, axis=1)


In [7]:
# trim df to only certain length
LENGTH = 6
fm_df = fdf[(fdf["expiry_length"] == LENGTH) | (fdf["expiry_length"] == LENGTH + 1)].copy()

In [8]:
# make ts event index
fm_df = fm_df.set_index('ts_event', inplace=False)

In [9]:
fm_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4524 entries, 2010-06-06 00:00:00+00:00 to 2025-07-31 00:00:00+00:00
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   rtype           4524 non-null   int64  
 1   publisher_id    4524 non-null   int64  
 2   instrument_id   4524 non-null   int64  
 3   open            4524 non-null   float64
 4   high            4524 non-null   float64
 5   low             4524 non-null   float64
 6   close           4524 non-null   float64
 7   volume          4524 non-null   int64  
 8   symbol          4524 non-null   object 
 9   contract_year   4524 non-null   int64  
 10  contract_month  4524 non-null   int64  
 11  expiry_length   4524 non-null   int64  
dtypes: float64(4), int64(7), object(1)
memory usage: 459.5+ KB


In [10]:
fm_df.tail(10)

,rtype,publisher_id,instrument_id,open,high,low,close,volume,symbol,contract_year,contract_month,expiry_length
ts_event,,,,,,,,,,,,
2025-07-27 00:00:00+00:00,35,1,42001025,3400.5,3415.1,3399.4,3415.1,7,GCG6,2026,2,7
2025-07-27 00:00:00+00:00,35,1,42028866,3394.0,3407.0,3394.0,3403.4,4,GCF6,2026,1,6
2025-07-28 00:00:00+00:00,35,1,42028866,3405.0,3408.4,3385.4,3385.4,16,GCF6,2026,1,6
2025-07-28 00:00:00+00:00,35,1,42001025,3414.1,3427.2,3385.2,3401.1,287,GCG6,2026,2,7
2025-07-29 00:00:00+00:00,35,1,42001025,3393.5,3414.3,3391.4,3410.3,176,GCG6,2026,2,7
2025-07-29 00:00:00+00:00,35,1,42028866,3379.6,3401.0,3379.6,3399.9,25,GCF6,2026,1,6
2025-07-30 00:00:00+00:00,35,1,42028866,3401.3,3402.0,3341.4,3344.7,23,GCF6,2026,1,6
2025-07-30 00:00:00+00:00,35,1,42001025,3414.1,3415.8,3349.2,3360.3,413,GCG6,2026,2,7
2025-07-31 00:00:00+00:00,35,1,42028866,3380.0,3380.0,3354.5,3354.5,2,GCF6,2026,1,6


In [ ]:
# construct front month dataframe (only take (LENGTH + 1) month if LENGTH month is unavailable)

# identify rows where expiry_length is LENGTH + 1 and it is duplicate
indices_with_correct_len = set(fm_df.loc[fm_df['expiry_length'] == LENGTH].index)
mask_to_drop = (fm_df['expiry_length'] == LENGTH + 1) & (fm_df.index.isin(indices_with_correct_len))
frdf = fm_df[~mask_to_drop].copy()

In [19]:
# determine transition dates (determines when to roll)
frdf['roll_marker'] = frdf['symbol'] != frdf['symbol'].shift(-1)
frdf.loc[frdf.index[-1], 'roll_marker'] = False
print(f"there are {sum(frdf['roll_marker'])} transitions")

there are 111 transitions


C:\Users\semic\AppData\Local\Temp\ipykernel_17876\1505604719.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frdf['roll_marker'] = frdf['symbol'] != frdf['symbol'].shift(-1)


In [22]:
frdf.iloc[-15:-5]

,rtype,publisher_id,instrument_id,open,high,low,close,volume,symbol,contract_year,contract_month,expiry_length,roll_marker
ts_event,,,,,,,,,,,,,
2025-07-14 00:00:00+00:00,35,1,42001025,3456.9,3468.1,3432.6,3436.0,118,GCG6,2026,2,7,False
2025-07-15 00:00:00+00:00,35,1,42001025,3439.3,3458.4,3411.8,3419.0,155,GCG6,2026,2,7,False
2025-07-16 00:00:00+00:00,35,1,42001025,3421.7,3469.0,3409.7,3437.0,234,GCG6,2026,2,7,False
2025-07-17 00:00:00+00:00,35,1,42001025,3429.6,3433.9,3399.3,3428.7,153,GCG6,2026,2,7,False
2025-07-18 00:00:00+00:00,35,1,42001025,3427.8,3450.5,3427.3,3440.7,135,GCG6,2026,2,7,True
2025-07-21 00:00:00+00:00,35,1,42028866,3464.7,3486.0,3464.7,3486.0,5,GCF6,2026,1,6,False
2025-07-22 00:00:00+00:00,35,1,42028866,3471.0,3510.0,3471.0,3510.0,5,GCF6,2026,1,6,False
2025-07-23 00:00:00+00:00,35,1,42028866,3464.0,3471.8,3464.0,3471.8,2,GCF6,2026,1,6,False
2025-07-24 00:00:00+00:00,35,1,42028866,3467.0,3467.0,3438.0,3445.3,4,GCF6,2026,1,6,False


In [31]:
fdf[fdf['ts_event'] == pd.to_datetime('2025-07-17', utc=True)]

,ts_event,rtype,publisher_id,instrument_id,open,high,low,close,volume,symbol,contract_year,contract_month,expiry_length
137173,2025-07-17 00:00:00+00:00,35,1,42000890,3427.2,3452.9,3422.8,3452.0,42,GCJ6,2026,4,9
137174,2025-07-17 00:00:00+00:00,35,1,42001967,3379.8,3379.9,3342.0,3372.7,3466,GCV5,2025,10,3
137176,2025-07-17 00:00:00+00:00,35,1,42025107,3366.3,3366.3,3329.9,3361.8,507,GCU5,2025,9,2
137178,2025-07-17 00:00:00+00:00,35,1,42020775,3313.8,3340.8,3313.8,3340.8,14,GCN5,2025,7,0
137179,2025-07-17 00:00:00+00:00,35,1,42011464,3502.2,3502.2,3479.8,3500.8,8,GCQ6,2026,8,13
137181,2025-07-17 00:00:00+00:00,35,1,19181,3461.6,3476.6,3461.6,3476.6,11,GCM6,2026,6,11
137183,2025-07-17 00:00:00+00:00,35,1,41271,3531.6,3533.6,3530.2,3530.2,4,GCZ6,2026,12,17
137186,2025-07-17 00:00:00+00:00,35,1,42001025,3429.6,3433.9,3399.3,3428.7,153,GCG6,2026,2,7
137199,2025-07-17 00:00:00+00:00,35,1,42008094,3351.7,3352.8,3314.3,3345.6,144765,GCQ5,2025,8,1
137205,2025-07-17 00:00:00+00:00,35,1,42021912,3374.4,3380.0,3359.8,3380.0,10,GCX5,2025,11,4


In [46]:
for i in range(5, 8):
    for j in range(1, 30):
        if sum(df.loc[df['ts_event'] == pd.to_datetime(f'2025-{i}-{j}', utc=True), 'instrument_id'] == 42028866) == 0:
            if pd.to_datetime(f'2025-{i}-{j}', utc=True).weekday() not in [5, 6]:
                print(f'Found matching date: 2025-{i}-{j}')


Found matching date: 2025-5-1
Found matching date: 2025-5-2
Found matching date: 2025-5-5
Found matching date: 2025-5-7
Found matching date: 2025-5-8
Found matching date: 2025-5-9
Found matching date: 2025-5-12
Found matching date: 2025-5-13
Found matching date: 2025-5-14
Found matching date: 2025-5-15
Found matching date: 2025-5-16
Found matching date: 2025-5-19
Found matching date: 2025-5-21
Found matching date: 2025-5-22
Found matching date: 2025-5-23
Found matching date: 2025-5-26
Found matching date: 2025-5-27
Found matching date: 2025-5-28
Found matching date: 2025-5-29
Found matching date: 2025-6-2
Found matching date: 2025-6-4
Found matching date: 2025-6-9
Found matching date: 2025-6-10
Found matching date: 2025-6-18
Found matching date: 2025-6-19
Found matching date: 2025-6-24
Found matching date: 2025-6-25
Found matching date: 2025-6-26
Found matching date: 2025-6-27
Found matching date: 2025-7-2
Found matching date: 2025-7-3
Found matching date: 2025-7-4
Found matching date:

In [29]:
fm_df.loc[pd.to_datetime('2025-07-18', utc=True)]

rtype                   35
publisher_id             1
instrument_id     42001025
open                3427.8
high                3450.5
low                 3427.3
close               3440.7
volume                 135
symbol                GCG6
contract_year         2026
contract_month           2
expiry_length            7
Name: 2025-07-18 00:00:00+00:00, dtype: object

In [17]:
# handling transition
# checkign if each point has a transition
sdf = fm_df[mask_to_drop].copy()
roll_indices = frdf.index[frdf['roll_marker']]
available_transition_contracts = len(roll_indices.intersection(sdf.index))
print(available_transition_contracts)

11


In [101]:
df[df['ts_event'] == pd.to_datetime("07-01-2025")]

,ts_event,rtype,publisher_id,instrument_id,open,high,low,close,volume,symbol


In [110]:
df[df['ts_event'] == pd.to_datetime("2025-07-02", utc=True)]

,ts_event,rtype,publisher_id,instrument_id,open,high,low,close,volume,symbol
136705,2025-07-02 00:00:00+00:00,35,1,42020775,3329.0,3362.0,3329.0,3350.0,23,GCN5
136706,2025-07-02 00:00:00+00:00,35,1,42026926,-153.4,-151.7,-154.3,-154.3,22,GCQ5-GCQ6
136707,2025-07-02 00:00:00+00:00,35,1,42066744,-11.6,-11.6,-12.0,-11.6,3,GCG6-GCH6
136708,2025-07-02 00:00:00+00:00,35,1,42001025,3428.0,3456.3,3421.0,3443.0,158,GCG6
136709,2025-07-02 00:00:00+00:00,35,1,42066068,-14.8,-14.6,-14.8,-14.8,43,GCV5-GCX5
136710,2025-07-02 00:00:00+00:00,35,1,42008094,3348.7,3376.9,3337.2,3360.3,123814,GCQ5
136711,2025-07-02 00:00:00+00:00,35,1,42037374,-23.7,-23.4,-24.0,-23.8,59,GCJ6-GCM6
136712,2025-07-02 00:00:00+00:00,35,1,42063221,3530.0,3530.0,3530.0,3530.0,2,GCV6
136713,2025-07-02 00:00:00+00:00,35,1,42010873,-55.2,-55.1,-55.8,-55.6,7530,GCQ5-GCZ5
136714,2025-07-02 00:00:00+00:00,35,1,42025107,3367.9,3391.0,3353.1,3378.0,332,GCU5


In [95]:
transition_indices

DatetimeIndex(['2010-07-01 00:00:00+00:00', '2010-09-01 00:00:00+00:00',
               '2010-11-01 00:00:00+00:00', '2011-03-01 00:00:00+00:00',
               '2011-07-01 00:00:00+00:00', '2011-09-01 00:00:00+00:00',
               '2011-11-01 00:00:00+00:00', '2012-03-01 00:00:00+00:00',
               '2012-05-01 00:00:00+00:00', '2012-11-01 00:00:00+00:00',
               '2013-01-01 00:00:00+00:00', '2013-03-01 00:00:00+00:00',
               '2013-05-01 00:00:00+00:00', '2013-07-01 00:00:00+00:00',
               '2013-11-01 00:00:00+00:00', '2014-05-01 00:00:00+00:00',
               '2014-07-01 00:00:00+00:00', '2015-05-01 00:00:00+00:00',
               '2015-07-01 00:00:00+00:00', '2015-09-01 00:00:00+00:00',
               '2016-03-01 00:00:00+00:00', '2016-07-01 00:00:00+00:00',
               '2016-09-01 00:00:00+00:00', '2016-11-01 00:00:00+00:00',
               '2017-03-01 00:00:00+00:00', '2017-05-01 00:00:00+00:00',
               '2017-09-01 00:00:00+00:00', '2017-1